In [1]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [2]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n!python3 -m pip install pandas\n!pip install chardet\n!pip install db-dtypes\n!pip install catboost\n")

In [3]:
#Importing Libraries
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import csv
import numpy as np

In [5]:
from google.cloud import bigquery
import scipy as sp
import statsmodels as sm
import statistics as stat
import sklearn
import patsy as pt
import chardet as ch
import click as cl
import cytoolz as cz
import dask
import decorator as dr
#import pyjson5 as py
import jsonschema as js
import tables
import db_dtypes
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

ModuleNotFoundError: No module named 'chardet'

In [6]:
global df3

In [7]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [8]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [9]:
query_job1 = bigquery_client.query(query)
query_job1

QueryJob<project=ironhacks-data, location=US, id=81b51a32-0084-4bff-8d05-04208759a04e>

In [10]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [11]:
query_job2 = bigquery_client.query(query)
query_job2

QueryJob<project=ironhacks-data, location=US, id=4380eb65-7aca-4c67-a5e4-0e4c4f3369e3>

In [12]:
!pip install db-dtypes
df1 = query_job2.to_dataframe()
df1.uu_id.nunique()

525

In [13]:
!pip install db-dtypes
df2 = query_job1.to_dataframe()
df2.uu_id.nunique()

525

In [14]:
df3 = pd.merge(df1,df2, on= ["uu_id","countyfips","tract","tract_name"])

In [15]:
#Expanding timeperiod to day, month and year
df3["time"] = pd.to_datetime(df3["timeperiod"], format= '%Y%m%d', errors = "coerce")
df3 =df3.drop("timeperiod", axis = 1)
df3["month"] = df3["time"].dt.month
df3["day"] = df3["time"].dt.day
str_col = ["tract_name", 'top_category_employer1', 'top_category_employer2',
       'top_category_employer3',"time"]
df3["uu_id_enc"] = df3["uu_id"].astype('category').cat.codes
#Encoding Strings
for col in str_col :
    df3[col] = df3[col].astype('category').cat.codes
int_col = list(set(df3.columns) -set(str_col)-set(["uu_id"]))
#dropping and replacing Nan, NaT   
df3 =df3.astype(str)
df3 = df3[df3["uu_id"] != np.nan]
df3 = df3.replace('NaT', np.nan)
df3 = df3.replace(str(np.nan),str(0))
#df3 = df3.dropna(subset = ["uu_id"], inplace=True)
for col in df3.columns:
    df3[col] = df3[col].str.replace(str("<NA>"),"0")
for col in int_col:
    df3[col] =df3[col].astype(float)
df = df3.copy()
df3.uu_id.nunique()

525

In [16]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [17]:
query_job3 = bigquery_client.query(query)
test_df = query_job3.to_dataframe()
#test_df["time"] = pd.datetime(2022-09-24, format = %Y%m%d)
test_df["day"] = 7
test_df["month"]=10

In [18]:
test_df.drop_duplicates()

,uu_id,week_number,day,month
0,5bf51fc2e162d6faf9e3cf79e4198378,44,7,10
1,420b44cc7e3f55d738df565421e59941,44,7,10
2,e39c66ecceec76ee8f9f811fa4a2d246,44,7,10
3,a90462cd11ae4e43144239bf7c4828a4,44,7,10
4,8b20a6749088c7ff1237983076ebfeaa,44,7,10
...,...,...,...,...
520,46c4f6c75e663b1ca82ea7994e6d83d3,44,7,10
521,1deebda501712e7595b531b8337bc31a,44,7,10
522,5a9758f65f001b6432ff31ff64a459d7,44,7,10
523,e8b3b95e93a6dc7dbb90f4e72e7ac065,44,7,10


In [19]:
test_df = pd.merge(test_df, df3[["uu_id","uu_id_enc"]], on="uu_id").drop(["uu_id"], axis=1).drop_duplicates()
feature_test_pred = test_df.copy()
test_df

,week_number,day,month,uu_id_enc
0,44,7,10,198.0
10,44,7,10,126.0
18,44,7,10,476.0
32,44,7,10,353.0
58,44,7,10,288.0
...,...,...,...,...
16684,44,7,10,144.0
16722,44,7,10,52.0
16757,44,7,10,192.0
16781,44,7,10,486.0


In [20]:
#df3 =df3.dropna()
li=[]
import statsmodels.api as sm
import statistics    
for k in df3.uu_id_enc.unique():
    temp=df3[df3.uu_id_enc == k]
    temp = temp.sort_values("week_number")
    df3_ = pd.concat([temp,test_df])
    df3_=df3_[["week_number","uu_id_enc","total_claims"]].drop_duplicates()
    #df3_["predicted_total_claims"]=0
    mod = sm.tsa.statespace.SARIMAX(np.array(temp["total_claims"]),
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 2),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
    try: 
        results = mod.fit()
    except IndexError:
        g = df3[df3.uu_id_enc==k]
        val= g[g.week_number==39]['total_claims'].mean()
    pred = results.get_prediction(start=40, end =40, dynamic=False)
    val = (pred.predicted_mean)
    test_df.loc[test_df.uu_id_enc==k,"total_claims"] = val
    li.append(pred.predicted_mean)
    pred_ci = pred.conf_int()
    #print(test_df.loc[test_df.uu_id_enc==k,col],val.predict(np.array(feature_test_pred)))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.51304D+00    |proj g|=  7.30584D-02

At iterate    5    f=  2.36879D+00    |proj g|=  3.32271D-01

At iterate   10    f=  2.23125D+00    |proj g|=  8.98104D-02

At iterate   15    f=  2.21721D+00    |proj g|=  5.25696D-02

At iterate   20    f=  2.21388D+00    |proj g|=  5.54572D-03

At iterate   25    f=  2.21384D+00    |proj g|=  5.29234D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     25     35      1     0     0   

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



At iterate    5    f=  3.03233D+00    |proj g|=  1.15997D-01

At iterate   10    f=  2.88260D+00    |proj g|=  6.29058D-02

At iterate   15    f=  2.88027D+00    |proj g|=  2.97506D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     18     24      1     0     0   2.686D-05   2.880D+00
  F =   2.8802681254677749     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.25756D+00    |proj g|=  2.76026D-01

At iter

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 This problem

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.37597D+00    |proj g|=  1.70996D-01

At iterate    5    f=  3.30868D+00    |proj g|=  6.41735D-02

At iterate   10    f=  3.15989D+00    |proj g|=  9.38311D-02

At iterate   15    f=  3.12582D+00    |proj g|=  1.44598D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     19     22      1     0     0   3.682D-06   3.126D+00
  F =   3.1258186841059019     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING 

...

 This problem is unconstrained.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 This problem is unconstrained.
 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood o


At iterate    5    f=  8.70039D-01    |proj g|=  4.12531D-02

At iterate   10    f=  8.22130D-01    |proj g|=  1.87927D-01

At iterate   15    f=  6.49813D-01    |proj g|=  2.32108D+00

At iterate   20    f=  2.68611D-01    |proj g|=  1.13381D+01

At iterate   25    f= -1.18926D-01    |proj g|=  5.13364D+01

At iterate   30    f= -3.26305D-01    |proj g|=  3.58205D+02

At iterate   35    f= -4.27752D-01    |proj g|=  1.97907D+02

At iterate   40    f= -5.06918D-01    |proj g|=  1.77156D+01

At iterate   45    f= -5.29426D-01    |proj g|=  4.84801D+00
  ys=-3.829E-12  -gs= 2.690E-10 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



At iterate   40    f=  3.09496D-01    |proj g|=  6.95454D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     41     77      1     0     0   6.911D-03   3.095D-01
  F =  0.30949581994679937     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.65857D+00    |proj g|=  3.71166D-01

At iterate    5    f=  3.55655D+00    |proj g|=  1.89288D-02

At iterate   10    f=  3.49707D+00    |proj g|=  2.67331D-02

At iter

 This problem is unconstrained.
/opt/homebrew/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 This problem is unconstrained.


In [21]:
test_df

,week_number,day,month,uu_id_enc,total_claims
0,44,7,10,198.0,1.171129e+02
10,44,7,10,126.0,-1.369146e+17
18,44,7,10,476.0,1.351638e+01
32,44,7,10,353.0,-2.228366e+00
58,44,7,10,288.0,1.079835e+01
...,...,...,...,...,...
16684,44,7,10,144.0,4.247554e+01
16722,44,7,10,52.0,3.280737e+01
16757,44,7,10,192.0,1.223219e+01
16781,44,7,10,486.0,1.566907e+00


In [22]:
test_df = pd.merge(test_df, df3[["uu_id","uu_id_enc"]], on="uu_id_enc")
submission_prediction_output= test_df[["uu_id","week_number","total_claims"]].drop_duplicates()
submission_prediction_output["total_claims"] = submission_prediction_output["total_claims"].fillna(0)
for k in submission_prediction_output.loc[submission_prediction_output["total_claims"]<=0]["uu_id"]:
    submission_prediction_output.loc[submission_prediction_output["uu_id"]==k,"total_claims"].value=df3[df3.uu_id==k]["total_claims"].mean()
submission_prediction_output.to_csv("submission_prediction_output.csv")
submission_prediction_output

,uu_id,week_number,total_claims
0,5bf51fc2e162d6faf9e3cf79e4198378,44,1.171129e+02
10,420b44cc7e3f55d738df565421e59941,44,-1.369146e+17
18,e39c66ecceec76ee8f9f811fa4a2d246,44,1.351638e+01
32,a90462cd11ae4e43144239bf7c4828a4,44,-2.228366e+00
58,8b20a6749088c7ff1237983076ebfeaa,44,1.079835e+01
...,...,...,...
16684,46c4f6c75e663b1ca82ea7994e6d83d3,44,4.247554e+01
16722,1deebda501712e7595b531b8337bc31a,44,3.280737e+01
16757,5a9758f65f001b6432ff31ff64a459d7,44,1.223219e+01
16781,e8b3b95e93a6dc7dbb90f4e72e7ac065,44,1.566907e+00


In [23]:
submission_prediction_output[submission_prediction_output.total_claims<=0]

,uu_id,week_number,total_claims
10,420b44cc7e3f55d738df565421e59941,44,-1.369146e+17
32,a90462cd11ae4e43144239bf7c4828a4,44,-2.228366e+00
755,d6a4947ec7c2a78bd32ba4e30f3bba53,44,-7.040092e+04
907,3934b01a1e31d0a6af96867e1afbf03c,44,-6.731355e+27
995,f25446d4d951867ca7b984ad17c6c1e0,44,-5.595119e+01
1196,b3a4cfbf22f45360e7fb11d1f8f86b9e,44,-3.625370e+00
3521,99c1cb11273b064dc48649886138eb8e,44,-4.400000e+01
5725,747f8bc2b0c8c0a04d29caa4cfe327d2,44,-2.600000e+01
6583,dfa86215568c869a2e94ce3e5fcf1661,44,-2.122008e+05
7357,aef08e5cd6bbf488ae574e9462f3e47b,44,-1.014179e+01


In [24]:
for k in submission_prediction_output.loc[submission_prediction_output["total_claims"]<=0]["uu_id"]:
    submission_prediction_output.loc[submission_prediction_output["uu_id"]==k,"total_claims"].value=df3[df3.uu_id==k]["total_claims"].mean()
submission_prediction_output[submission_prediction_output.total_claims<=0]

,uu_id,week_number,total_claims
10,420b44cc7e3f55d738df565421e59941,44,-1.369146e+17
32,a90462cd11ae4e43144239bf7c4828a4,44,-2.228366e+00
755,d6a4947ec7c2a78bd32ba4e30f3bba53,44,-7.040092e+04
907,3934b01a1e31d0a6af96867e1afbf03c,44,-6.731355e+27
995,f25446d4d951867ca7b984ad17c6c1e0,44,-5.595119e+01
1196,b3a4cfbf22f45360e7fb11d1f8f86b9e,44,-3.625370e+00
3521,99c1cb11273b064dc48649886138eb8e,44,-4.400000e+01
5725,747f8bc2b0c8c0a04d29caa4cfe327d2,44,-2.600000e+01
6583,dfa86215568c869a2e94ce3e5fcf1661,44,-2.122008e+05
7357,aef08e5cd6bbf488ae574e9462f3e47b,44,-1.014179e+01


In [25]:
for k in submission_prediction_output.loc[submission_prediction_output["total_claims"]<=0]["uu_id"]:
    submission_prediction_output.loc[submission_prediction_output["uu_id"]==k,"total_claims"]=df3[df3.uu_id==k]["total_claims"].mean()
submission_prediction_output[submission_prediction_output.total_claims<=0]

,uu_id,week_number,total_claims


In [26]:
for k in submission_prediction_output.loc[submission_prediction_output["total_claims"]<=0]["uu_id"]:
    submission_prediction_output.loc[submission_prediction_output["uu_id"]==k,"total_claims"]=df3[df3.uu_id==k]["total_claims"].mean()
submission_prediction_output[submission_prediction_output.total_claims<=0]
submission_prediction_output.to_csv("submission_prediction_output.csv")